https://hd.stheadline.com/news/realtime/fin/2391104/%E5%8D%B3%E6%99%82-%E9%87%91%E8%9E%8D-%E6%B8%A3%E6%89%93%E4%BF%A1%E7%94%A8%E5%8D%A1%E7%88%86%E5%A4%A7%E8%A6%8F%E6%A8%A1%E7%9B%9C%E7%94%A8-%E5%AE%A2%E6%9C%8D%E5%94%94%E9%80%9A-%E6%89%8B%E6%A9%9F%E6%AD%BBApp-%E6%B8%A3%E6%89%93%E6%99%9A%E4%B8%8A%E6%94%B9%E5%AE%89%E6%8E%92-%E5%B0%87%E4%B8%BB%E5%8B%95%E9%80%80%E6%AC%BE-%E7%84%A1%E8%AD%89%E6%93%9A%E8%B3%87%E6%96%99%E5%A4%96%E6%B4%A9

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import sys
import time
import requests
from itertools import cycle
from datetime import datetime
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import undetected_chromedriver as uc
from datetime import timedelta, date, datetime

In [2]:
#functions
def click_thread(idx):
    #click thread
    auto_xpath = f'//*[@id="leftPanel"]/div[2]/div[{idx + 1}]/div/div[2]/a[1]'
    try:
        driver.find_element(By.XPATH, auto_xpath).click()
        #print(idx)
    except:
        #print("skip")
        pass
    time.sleep(0.3)

In [3]:
#Open pandas
try:
    df_threads = pd.read_excel('LIHKG_Thread_Level.xlsx')
except:
    df_threads = pd.DataFrame(columns=["Thread UID",
                                       "Keyword",
                                       "Reference Bank",
                                       "Fetch Datetime",
                                       "Thread Post Datetime",
                                       "Thread Author",
                                       "Thread Lastest Comment Time(Thread info)",
                                       "Thread Lastest Comment Time(Fetch inside comment)",
                                       "Thread Title",
                                       "Thread Number of Page",
                                       "Thread Theme",
                                       "Thread Likes",
                                       "Thread Dislike",
                                       "Last Fetch Comment Floor Number"])

#Get Distinct value by last row
df_distinct_thread_by_latest = df_threads.drop_duplicates(subset=['Keyword',
                                                                  'Thread Author',
                                                                  'Thread Title',
                                                                  'Thread Theme'], keep='last')

try:
    df_comments = pd.read_excel('LIHKG_Comment_Level.xlsx')

except:
    df_comments = pd.DataFrame(columns=["Thread UID",
                                        "Comment UID",
                                        "Fetch Datetime",
                                        "Comment Floor Number",
                                        "Comment Author",
                                        "Comment Date",
                                        "Comment Time",
                                        "Comment Likes",
                                        "Comment Dislike",
                                        "Comment Replies",
                                        "Comment Context"])   

df_keyword = pd.read_excel('LIHKG_Keyword.xlsx', index_col=None)
keyword_list = df_keyword["Keyword"].values.tolist()
ref_bank_list = df_keyword["Reference Bank"].values.tolist()
df_keyword

,Keyword,Reference Bank
0,渣打信用卡,Standard Chartered
1,揸兜信用卡,Standard Chartered


In [4]:
#set driver to uc driver
driver = uc.Chrome()

#starts of the loop

for keyword_row_num, keyword in enumerate(keyword_list):
    ref_bank = ref_bank_list[keyword_row_num]
    fetch_datetime = datetime.now().strftime("%Y%m%d-%H%M%S")
    keyword_uid = "LI-"+str(fetch_datetime)+"-"+ str(keyword_row_num).zfill(8)
    
    #get link
    driver.get(f"https://lihkg.com/search?q={keyword}&sort=desc_create_time&type=thread")
    time.sleep(4)
    
    #get html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #click left panel for scrolling
    driver.find_element(By.XPATH, '//*[@id="leftPanel"]/div[1]/ul/li[2]/a').click()

    #scroll to the end (leftpanel)
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html)
        checker = soup.find_all("div",{"class":"_21IQKhlBjN2jlHS_TVgI3l"})
        try:
            if len(checker) == previous_checker:
                break
        except:
            pass
        
        previous_checker = len(checker)
        
        try:
            driver.find_element(By.CLASS_NAME, '_33r1FGqGJZF-fM1VZm7mhN').text
            time.sleep(2)
            break
        except:
            pass

        for i in range(20):
            ActionChains(driver)\
                .send_keys(" ")\
                .perform()
            time.sleep(0.1)
    
    #get latest html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #get thread html info
    threads = soup.find("div",{"class":"qoAmEqNpZRLf2KVKZ8DsC"})
    for thread_idx, i in enumerate(threads):
        #click thread
        click_thread(thread_idx)
        
        #create thread_uid by reverse order
        thread_uid = keyword_uid + "-" +str(len(threads) - (thread_idx + 1)).zfill(8)
        
        #get thread info
        try:
            try:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 A0jheqYUBHNW93KykXKEH"}).text
            except:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 jj_3ZDzjtPixL1b2KTcpS"}).text

            thread_lastest_comment_time = i.find("span", {"class":"_37XwjAqVHtjzqzEtybpHrU"}).text
            
            ###
            #check day or hr or month function & break it when the time is not in range
            #
            #
            ###

            thread_title = i.find("span", {"class":"_20jopXBFHNQ9FUbcGHLcHH"}).text
            thread_page = i.find("div", {"class":"_26oEXjfUS_iHzbxYcZE6bD"}).text.split(" ")[0]
            thread_theme = i.find_all("a")[1].text
            thread_total_like = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[0].get_attribute('data-score')
            thread_total_dislike = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[1].get_attribute('data-score')

        except:
            pass
        
        #Locate the thread in excel
        try:
            floor_number_loc = df_distinct_thread_by_latest.loc[(df_distinct_thread_by_latest['Keyword'] == keyword)
                                                                & (df_distinct_thread_by_latest['Thread Author'] == thread_author) 
                                                                & (df_distinct_thread_by_latest['Thread Title'] == thread_title)
                                                                & (df_distinct_thread_by_latest['Thread Theme'] == thread_theme)]
            floor_number_pointer = floor_number_loc["Last Fetch Comment Floor Number"].values[0]
        except:
            floor_number_pointer = '#0'
            
        #current time
        fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)

        #loop to the end of pages
        for click in range(int(thread_page)):
            '''
            ###
            very rare case when occur a thread that have very longgggg
            context in the first post, the f5 detect will not working.
            Thus it will skip the scrolling and fetch data directly.
            ###
            '''
            try:
                find_f5 = driver.find_element(By.CLASS_NAME, '_1PhR8JHkMcET5QT2PtCA3T')
                actions = ActionChains(driver)
                actions.move_to_element(find_f5).perform()
                time.sleep(1)
            except:
                try:
                    find_f5 = driver.find_element(By.XPATH, f'//*[@id="page-{str(click+1)}"]/div[3]/a').click()
                except:
                    for j in range(20):
                        ActionChains(driver).send_keys(" ").perform()

                        
        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)
        
        #start of fetching comments data
        all_comment = soup.find_all("div",{"class":"_36ZEkSvpdj_igmog0nluzh"})
        for comment_idx , comment in enumerate(all_comment):
            
            comment_floor_number = comment.find_all("span")[0].text
            comment_uid = thread_uid + "-" + str(comment_floor_number.replace("#", "")).zfill(8)
            comment_author = comment.find("span",{"class":"ZZtOrmcIRcvdpnW09DzFk"}).text

            #incase the comment is folded due to not enough info
            try:
                comment_datetime = comment.find("span",{"class":"Ahi80YgykKo22njTSCzs_"})
                new_comment_datetime = str(comment_datetime).split("\"")[5]
                comment_date = new_comment_datetime.split(" ")[0]
                comment_time = new_comment_datetime.split(" ")[1]

                comment_like = comment.find_all("label")[1].text
                comment_dislike = comment.find_all("label")[3].text
            except:
                comment_datetime = ""
                comment_date = ''
                comment_time = ''
                comment_like = 0
                comment_dislike = 0

            try:
                comment_replies = comment.find_all("label")[5].text
            except:
                comment_replies = 0

            try:
                comment_context = comment.find("div",{"class":"_2cNsJna0_hV8tdMj3X6_gJ"}).text
            except:
                comment_context = ''
            
            #find earliest time
            if comment_idx == 0:
                thread_post_datetime = new_comment_datetime
            
            #get the lastest comment time
            latest_comment_time = new_comment_datetime
                
            #get lastest floor number in this fetch
            lastest_floor_number = comment_floor_number
            
            #current time
            fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            #compare the latest floor number
            if int(floor_number_pointer.replace("#", "")) < int(comment_floor_number.replace("#", "")):
                df_comments.loc[len(df_comments)]= [thread_uid,
                                                     comment_uid,
                                                     fetch_datetime,
                                                     comment_floor_number,
                                                     comment_author,
                                                     comment_date,
                                                     comment_time,
                                                     int(comment_like),
                                                     int(comment_dislike), 
                                                     int(comment_replies), 
                                                     comment_context]
        #save thread into to dataframe
        df_threads.loc[len(df_threads)]= [thread_uid,
                                          keyword,
                                          ref_bank,
                                          fetch_datetime,
                                          thread_post_datetime,
                                          thread_author, 
                                          thread_lastest_comment_time,
                                          latest_comment_time,
                                          thread_title,
                                          int(thread_page),
                                          thread_theme, 
                                          int(thread_total_like), 
                                          int(thread_total_dislike),
                                          lastest_floor_number]
        
        break
    break

driver.quit()

In [5]:
df_threads

,Thread UID,Keyword,Reference Bank,Fetch Datetime,Thread Post Datetime,Thread Author,Thread Lastest Comment Time(Thread info),Thread Lastest Comment Time(Fetch inside comment),Thread Title,Thread Number of Page,Thread Theme,Thread Likes,Thread Dislike,Last Fetch Comment Floor Number
0,LI-20230103-213713-00000000-00000036,渣打信用卡,Standard Chartered,2023-01-03 21:37:37,2022年12月9日 09:37:13,GME散戶（暴發）,3 小時前,2023年1月3日 17:41:13,突發！張渣打信用卡俾人碌左萬鳩幾蚊！！,12,財經台,680,21,#21
1,LI-20230103-213737-00000001-00000002,揸兜信用卡,Standard Chartered,2023-01-03 21:40:31,2022年12月6日 21:04:25,神宮寺,27 日前,2022年12月7日 15:17:16,揸兜信用卡比人碌到錢,41,財經台,33,465,#1001
2,LI-20230104-002956-00000000-00000053,渣打信用卡,Standard Chartered,2023-01-04 00:30:25,2022年12月9日 09:37:13,GME散戶（暴發）,6 小時前,2023年1月3日 17:41:13,突發！張渣打信用卡俾人碌左萬鳩幾蚊！！,12,財經台,680,21,#282


In [6]:
df_comments

,Thread UID,Comment UID,Fetch Datetime,Comment Floor Number,Comment Author,Comment Date,Comment Time,Comment Likes,Comment Dislike,Comment Replies,Comment Context
0,LI-20230103-213713-00000000-00000036,LI-20230103-213713-00000000-00000036-00000001,2023-01-03 21:37:37,#1,GME散戶（暴發）,2022年12月9日,09:37:13,680,21,0,\n\n\n\n\n\n覆完之後入app 睇\n我根本冇呢張尾數既卡\n做乜鳩 咩事？？
1,LI-20230103-213713-00000000-00000036,LI-20230103-213713-00000000-00000036-00000002,2023-01-03 21:37:37,#2,洪為民請命,2022年12月9日,09:38:01,75,308,15,坐撚咗兩日？無睇新聞？
2,LI-20230103-213713-00000000-00000036,LI-20230103-213713-00000000-00000036-00000003,2023-01-03 21:37:37,#3,99912345,2022年12月9日,09:39:06,83,0,0,無事 呢幾日算係正常
3,LI-20230103-213713-00000000-00000036,LI-20230103-213713-00000000-00000036-00000004,2023-01-03 21:37:37,#4,那日的笑容,2022年12月9日,09:55:38,77,20,10,呢鋪渣打會唔會破產清盤?
4,LI-20230103-213713-00000000-00000036,LI-20230103-213713-00000000-00000036-00000005,2023-01-03 21:37:37,#5,奧斯卡PS3,2022年12月9日,09:55:42,219,3,1,根本冇呢張尾數既卡
...,...,...,...,...,...,...,...,...,...,...,...
277,LI-20230104-002956-00000000-00000053,LI-20230104-002956-00000000-00000053-00000278,2023-01-04 00:30:25,#278,Iamhippoman,2022年12月27日,20:04:35,0,0,0,我都係垃撚圾銀行，永不錄用
278,LI-20230104-002956-00000000-00000053,LI-20230104-002956-00000000-00000053-00000279,2023-01-04 00:30:25,#279,敲母太郎,2022年12月28日,00:00:34,0,0,0,咁就唔知了\n都係舖頭同個畀人盜用us$1嘅客講再傳到我度
279,LI-20230104-002956-00000000-00000053,LI-20230104-002956-00000000-00000053-00000280,2023-01-04 00:30:25,#280,tomtomyo,2022年12月28日,10:19:48,0,0,1,可能遲些少先退，不過最緊要係有得退錢
280,LI-20230104-002956-00000000-00000053,LI-20230104-002956-00000000-00000053-00000281,2023-01-04 00:30:25,#281,派特列圖,2023年1月1日,13:20:20,0,0,1,今日check完都仲未退


In [7]:
#output data
df_threads.to_excel('LIHKG_Thread_Level.xlsx',encoding= 'utf-8-sig',index = None) 
df_comments.to_excel('LIHKG_Comment_Level.xlsx',encoding= 'utf-8-sig',index = None) 

C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
